# Task 3 - Cluster Analysis

*To better understand what typical charging sessions look like, carry out a cluster
analysis to provide management with a succinct report of archetypical charging events. Think of an
appropriate trade-off between explainability and information content and try to come up with names
for these clusters. What is the value of identifying different types of charging sessions?*

In [ ]:
# Import all necessary libraries

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sklearn
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from sklearn.base import ClusterMixin
import plotly.express as px
from scipy.cluster.hierarchy import dendrogram, linkage
from tqdm.notebook import tqdm
from itertools import *
import ast
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

RAND = 10

In [ ]:
# open dataset
data_clusters = pd.DataFrame(pd.read_csv("../datasets/df_charging_sessions_with_weather.csv"))
data_clusters['connectionTime'] = pd.to_datetime(data_clusters['connectionTime'], utc=True).dt.tz_convert('America/Los_Angeles')
data_clusters['disconnectTime'] = pd.to_datetime(data_clusters['disconnectTime'], utc=True).dt.tz_convert('America/Los_Angeles')
data_clusters['doneChargingTime'] = pd.to_datetime(data_clusters['doneChargingTime'], utc=True).dt.tz_convert('America/Los_Angeles')

In [ ]:
data_clusters.head(2)

In [ ]:
print('\033[1m' + '\033[01;04;38;05;23m' + 'DATA ON THE DATAFRAME' + '\033[01;00;38;05;23m')
print()
print('\033[1m' + '\033[01;38;05;30m' + '1. Let\'s examine the dataframe information:' + '\033[0m')
print()
display(data_clusters.info())  # Display information about all columns
print()
print('\033[1m' + '\033[01;38;05;30m' + '2. Check for complete duplicates in the data:')
duplicate_rows = data_clusters.drop(columns=['userInputs']).duplicated().sum()
display(duplicate_rows) # Checking for complete duplicates
print()
print('\033[1m' + '\033[01;38;05;30m' + '3. Let\'s output descriptive statistics:')
display(data_clusters.describe().round(2))  # Display descriptive statistics for all columns

In [ ]:
# Look at the user input 
row_data = data_clusters['userInputs'].iloc[63882]
row_data_1 = data_clusters['userInputs'].iloc[1]
print(row_data)
print(row_data_1)

In the 'userInputs' column, there are empty lists and lists with data. Let's see how many empty lists we have.

In [ ]:
# Function that first checks and converts a string to a list
def is_empty(lst):
    if isinstance(lst, str):  # Check if the element is a string
        try:
            lst = ast.literal_eval(lst)  # Convert the string to a list
        except (ValueError, SyntaxError):
            return False  
    return not lst
    
# Applying the function to each element in the 'userInputs' column
empty_counts = data_clusters['userInputs'].apply(is_empty).sum()
total_rows = len(data_clusters)

# Calculating the percentage of empty lists
empty_percentage = (empty_counts / total_rows) * 100

print(f"Number of empty lists: {empty_counts} from {total_rows}")
print(f"Percentage of empty lists: {empty_percentage:.2f}%")

#### Data Preparation for Cluster Analysis

For clustering we use only numerical features, but our data contains some categorical features. So the first step will be the data preparation. 

##### Deletion
- id - delete, because number of the record doesn't influence on the quality of the charging session   
- userID, stationID - delete, because we are not going to analyse the particolar users or stations, we are looking for general patterns 
- sessionID - redundant for clustering since each session is unique
- spaceID - we have kept siteID to see charging in private and public places, but we haven't information to analyse particular places because their names are encoded                      
- userInputs - we have 27% missing data, so to use presented data for clustering we will have to fill Nan with new values, but this could potentially distort the results of clustering, especially in algorithms sensitive to the distances between data points, such as K-means, hierarchical clustering

##### Transformation
- connectionTime - extract time, date, month and year
- disconnectTime - extract time and date (we assumed that month and year are the same with connection, so we already have this information)
- receive final plugged time (disconnectTime - connectionTime)
- doneChargingTime - extract charging time for session (doneChargingTime - connectionTime) to receive real charging time
- useless_parking - calculating parking time without charging (plugged time - charging time for session)

##### Encoding into a numerical feature    
- weather                            

In [ ]:
# Deletion
data_clusters = data_clusters.drop(columns=['id', 'userID', 'sessionID', 'userInputs',
                                            'spaceID', 'stationID'])
#check
data_clusters.info()

In [ ]:
# Calculating the plugging duration in minutes
data_clusters['charging_duration_disconnect'] = (
    (data_clusters['disconnectTime'] - data_clusters['connectionTime']).dt.total_seconds() / 60
).astype(int)

# Calculating the charging duration in minutes
data_clusters['charging_duration_done'] = (
    (data_clusters['doneChargingTime'] - data_clusters['connectionTime']).dt.total_seconds() / 60
).astype(int)

# Calculating parking time without charging
data_clusters['useless_parking'] = (
    (data_clusters['charging_duration_disconnect'] - data_clusters['charging_duration_done']))

data_clusters.info()

In [ ]:
# Extracting components from 'connectionTime' 
data_clusters['connection_year'] = data_clusters['connectionTime'].dt.year  # Extracting and adding year as a number
data_clusters['connection_month'] = data_clusters['connectionTime'].dt.month  # Extracting and adding month as a number
data_clusters['connection_day'] = data_clusters['connectionTime'].dt.day  # Extracting and adding day as a number
data_clusters['connection_hour'] = data_clusters['connectionTime'].dt.hour  # Extracting and adding hour as a number

# Extracting components from 'disconnectTime' 
data_clusters['disconnect_time'] = data_clusters['disconnectTime'].dt.hour  
data_clusters['disconnect_date'] = data_clusters['disconnectTime'].dt.day  

# Deletion 'connectionTime' and disconnectTime' 
data_clusters = data_clusters.drop(columns=['connectionTime', 'disconnectTime', 'doneChargingTime'])

#check
data_clusters.info()

Now we will observe the data to see if we have outliers which could influence on the clustering results.

#### Outliers

For each feature, we will construct pairs of histograms and box plots to more closely examine the distributions and identify outliers.

In [ ]:
# Function to count a share of outliers
def get_outliers(col: pd.Series, iqr_coeff=1.5):
    # Calculate the first and third quartiles of the column
    q25, q75 = col.quantile([.25, .75])
    
    # Calculate the interquartile range 
    distr_iqr = q75 - q25
    
    # Define the whiskers of the data, which are considered outliers
    whiskers = pd.Series([q25 - iqr_coeff * distr_iqr, q75 + iqr_coeff * distr_iqr])
    
    # Clip the whiskers to handle cases where whiskers extend beyond the range of the data
    whisker_low, whisker_upp = whiskers.clip(lower=col.min(), upper=col.max())
    
    # Calculate the ratio of outliers
    outliers_ratio = ((col > whisker_upp).sum() + (col < whisker_low).sum()) / len(col)

    return outliers_ratio

In [ ]:
# Function to plot the data + share of outliers
def get_eda(df: pd.DataFrame, outliers=True, iqr_coeff=1.5):
    '''
    Plots histograms and boxplot diagrams for all numeric features 

    Parameters:
    - df: The DataFrame containing the data.
    - outliers: A boolean flag to indicate whether to calculate and display outlier percentage.
    - iqr_coeff: The coefficient to calculate the IQR range for outlier detection.
    '''
    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=['number']).columns
    num_features = len(numeric_cols)
    
    # Adjusting the figsize for better readability
    fig, axs = plt.subplots(num_features, 2, figsize=(12, num_features * 4))  # Increased figsize for better fit
    
    # Define a medium font size for all text elements
    medium_font_size = 10
    plt.rcParams.update({'font.size': medium_font_size})

    # Create plots for each numeric column
    for idx, col in enumerate(numeric_cols):
        # Histogram
        axs[idx, 0].hist(df[col], bins=20, color='#35c0cd', edgecolor='black')
        axs[idx, 0].set_ylabel('Count', fontsize=medium_font_size)
        axs[idx, 0].set_xlabel(f'{col}', fontsize=medium_font_size)
        axs[idx, 0].tick_params(labelsize=medium_font_size)

        # Box plot
        box = axs[idx, 1].boxplot(df[col], vert=True, widths=0.5, patch_artist=True, boxprops=dict(facecolor="#35c0cd"))
        axs[idx, 1].tick_params(labelsize=medium_font_size)

        # Calculate and display the share of outliers if required
        if outliers:
            outlier_perc = get_outliers(df[col], iqr_coeff=iqr_coeff)
            axs[idx, 1].set_title(f'{col} Outliers: {outlier_perc:.1%}', fontsize=medium_font_size)
        else:
            axs[idx, 1].set_title(f'{col} Boxplot', fontsize=medium_font_size)

    # Adjust the layout of the plots to prevent overlapping
    plt.tight_layout()
    plt.show()

In [ ]:
get_eda(df=data_clusters)

In [ ]:
# Let's check also a standard deviation
data_clusters.describe()

After studying the graphs, let's describe the features in more detail.

- kWhDelivered: The majority of values are concentrated around zero with a rapid decline in frequency as the value increases. High values are rare and can be considered anomalous or outliers.

- siteID: This feature has two categories, with no outliers.

- temperature: The distribution of temperatures seems normal or close to normal with a slight skew to the left, indicating the presence of colder temperature values. Also there are some extreme temperature conditions.

- pressure: The distribution has a shape close to normal, the presence of many separate points below the lower "whisker" indicates the presence of outliers in the data.

- windspeed: There is a clear declining trend: the higher the wind speed, the fewer observations. The distribution is skewed to the right, with a long tail extending to higher speeds. The diagram also shows a significant number of outliers above the upper "whisker," indicating the presence of observations with unusually high wind speeds.

- precipitation: The bulk of the precipitation data is concentrated around the value of 0, suggesting that in most cases precipitation is absent or minimal. There are many points above the upper "whisker" on the box plot, representing outliers. This may indicate rare events of heavy precipitation.

- charging_duration_disconnect: 75% of observations have a low duration of charging before disconnection. The "whiskers" of the box plot extend significantly beyond the upper quartile, showing the spread of the remaining 25% of the data. The box plot also shows individual points that are located significantly above the upper "whisker." These points represent outliers.

- charging_duration_done: The majority of charging sessions have a very short duration. There is a sharp drop in frequency with increasing charging duration, and the data show a right-skewed distribution with a long tail, indicating the presence of a small number of cases with lengthy charging.

- useless_parking - also concentrated aroud zero, many outliers

- connection_year - distribution between 4 values, 4 peaks visible.

- connection_month, connection_day, disconnect_date: The distribution of observations is quite normal even with some variations in frequency between different months and dates of the month.

- connection_hour: The distribution shows a peak at the beginning of the day (or at the end of the previous one), which may indicate increased connection activity at this time. 

- disconnect_time: The most common time of disconnection is between approximately 15 and 18 hours. There are several points representing outliers that are located below the lower "whisker," possibly indicating disconnections occurring at unusually early hours.

The features with much outliers an big standard deviation are: kWhDelivered, charging_duration_done, charging_duration_disconnect, useless_parking. Let's delete outliers because they could influence on clustering algorythms. 

Also we suggested to delete the features 'precipitation' (not so much information, precipitation is absent or minimal) and 'windspeed' (distribution is skewed, significant number of outliers), 'connection_day' and 'disconnection_day' (we concentrate on seasonal patterns, for this month and temperature are enough).

In [ ]:
# delete some features 
data_clusters = data_clusters.drop(['windspeed', 'precipitation', 
                                    'connection_day', 'disconnect_date'], 
                                   axis=1)

In [ ]:
# check
data_clusters.info()

In [ ]:
# Function to remove outliers
def remove_outliers(data, column):
    Q3, Q1 = np.nanpercentile(data[column], [75, 25])
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    lower_bound = Q1 - 1.5 * IQR
    filtered_data = data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]
    
    return filtered_data

In [ ]:
data_clusters_outliers = remove_outliers(data=data_clusters, column='kWhDelivered')
data_clusters_outliers = remove_outliers(data=data_clusters, column='charging_duration_disconnect')
data_clusters_outliers = remove_outliers(data=data_clusters, column='charging_duration_done')
data_clusters_outliers = remove_outliers(data=data_clusters, column='useless_parking')

In [ ]:
# check
data_clusters_outliers.describe()

In [ ]:
get_eda(df=data_clusters_outliers)

In [ ]:
row_count = len(data_clusters)
clean_row_count = len(data_clusters_outliers)
deleted_rows = row_count - clean_row_count
deleted_fraction = deleted_rows / row_count * 100
print(f"Number of rows after cleaning: {clean_row_count}")
print(f"Fraction of rows deleted: {deleted_fraction:.2f}%")

In the graph, we can see that we have smoothly removed the outliers. We deleted around 1%, targeting only very rare cases, primarily because the algorithms we are going to use are sensitive to outliers. Let's also check the data for correlation, as this could also influence the clustering results.

##### Сorrelation 

In [ ]:
# Correlation checking

# Create a color map 
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Create a figure with two subplots 
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))  

# Create a mask for the upper triangle for the first correlation matrix
mask_corr1 = np.triu(np.ones_like(data_clusters.corr(numeric_only=True), dtype=bool))

# Draw the heatmap for the first correlation matrix
sns.heatmap(data_clusters.corr(numeric_only=True), mask=mask_corr1, cmap=cmap,
            vmax=1, vmin=-1, square=True, linewidths=.7, cbar_kws={"shrink": .8}, ax=ax1)
ax1.set_title('Correlation Matrix (with outliers).\nPearson coefficients.', fontsize=12)
ax1.tick_params(axis='x', labelsize=8)
ax1.tick_params(axis='y', labelsize=8)

# Create a mask for the upper triangle for the second correlation matrix
mask_corr2 = np.triu(np.ones_like(data_clusters_outliers.corr(numeric_only=True), dtype=bool))

# Draw the heatmap for the second correlation matrix
sns.heatmap(data_clusters_outliers.corr(numeric_only=True), mask=mask_corr2, cmap=cmap,
            vmax=1, vmin=-1, square=True, linewidths=.7, cbar_kws={"shrink": .8}, ax=ax2)
ax2.set_title('Correlation Matrix (without outliers).\nPearson coefficients.', fontsize=12)
ax2.tick_params(axis='x', labelsize=8)
ax2.tick_params(axis='y', labelsize=8)

# Adjust the color bar for the second heatmap
cbar2 = ax2.collections[0].colorbar
cbar2.ax.tick_params(labelsize=8)

# Adjust the layout of the figure to prevent overlapping of plots
plt.tight_layout()
plt.show()

There is no big difference in the sense of correlation between the data with outliers and without them. It decresed a bit between features useless_parking and charging_duration_disconnect because we deleted some rare cases of long charging. 

In [ ]:
data_clusters_outliers.corr(numeric_only=True)

There are some features with medium correlation for example charging_duration_disconnect and charging_duration_done (correlation=0.66), charging_duration_done and kWhDelivered	(correlation=0.54). They are also important for clustering because describe quality of charging sessions. Correlation could give biases to the clustering results. 

#### Encoding

In [ ]:
# Encoding categorical features
data_new = data_clusters.copy()
encoder = OrdinalEncoder()
encoded_columns = encoder.fit_transform(data_new[['weather']])
data_new[['weather']] = encoded_columns

#check
data_new.head(2)

In [ ]:
# Do the same for the data without outliers
data_no_outliers = data_clusters_outliers.copy()
encoded_columns = encoder.fit_transform(data_no_outliers[['weather']])
data_no_outliers[['weather']] = encoded_columns

data_no_outliers.head(1)

#### Scaling

In [ ]:
# Train the scaler on the features and scale them
scaler = StandardScaler()
scaler.fit(data_new)
X_scaled = scaler.transform(data_new)

# Convert the scaled data back into a DataFrame
X_scaled = pd.DataFrame(X_scaled, columns=data_new.columns, index=data_new.index)

# Display the first few rows of the scaled features
print(X_scaled.head(1))

In [ ]:
# Data without outliers
scaler.fit(data_no_outliers)
X_scaled_no_outliers = scaler.transform(data_no_outliers)

# Convert the scaled data back into a DataFrame
X_scaled_no_outliers = pd.DataFrame(X_scaled_no_outliers, columns=data_no_outliers.columns, index=data_no_outliers.index)

# Display the first few rows of the scaled features
print(X_scaled_no_outliers.head(1))

#### Data Dimension reduction

In [ ]:
# Data with outliers

# Fit PCA on the scaled data
pca = PCA().fit(X_scaled)

# Create a figure 
plt.figure(figsize=(6, 3))  

x = np.arange(1, len(pca.explained_variance_ratio_)+1, 1)

# Plot the cumulative sum of the explained variance ratio to show 
# how much variance is explained by each additional component
plt.plot(x, np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='--', color='b')
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.6, 0.87, '95% cut-off threshold', color = 'red', fontsize=8)
plt.grid(axis='x')
plt.title('PCA with outliers', fontsize=14)
plt.xlabel('number of components', fontsize=8)
plt.ylabel('cumulative explained variance', fontsize=8)
plt.show()

In [ ]:
# Perform PCA on the scaled data specifying 10 components to keep
pca = PCA(n_components=10).fit(X_scaled)

# Transform the scaled data 
components = pca.fit_transform(X_scaled)

# Calculate the total explained variance in percentage
total_var = pca.explained_variance_ratio_.sum() * 100

# Get the number of components from the PCA model
n_components = len(pca.explained_variance_ratio_)

# Create labels for the scatter matrix plot, including the percentage of variance for each component
labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca.explained_variance_ratio_ * 100)
}

# Create a scatter matrix of the PCA components 
fig = px.scatter_matrix(
    components,
    dimensions=range(n_components),
    labels=labels,
    title=f'Total Explained Variance: {total_var:.2f}%',
)

fig.update_traces(marker_size=1, diagonal_visible=False)

fig.update_layout(
    font=dict(size=8, color='black'),
    autosize=False,
    width=1100,
    height=1100,
)

fig.show()

In [ ]:
pca = PCA(n_components=10, random_state=RAND)
X_embedding_pca_10 = pca.fit_transform(X_scaled)

fig = px.scatter_3d(
    X_embedding_pca_10, x=0, y=1, z=2,
    labels={'color': 'species'}
)
fig.update_traces(marker_size=2)
fig.show();

Visually, it's difficult to determine the clusters; the data points are densely grouped, and there are no natural gaps between the points.

Let's do the same for the data without outliers.

In [ ]:
# Data without outliers
pca_1 = PCA().fit(X_scaled_no_outliers)
plt.figure(figsize=(6, 3))  
x = np.arange(1, len(pca_1.explained_variance_ratio_)+1, 1)
plt.plot(x, np.cumsum(pca_1.explained_variance_ratio_), marker='o', linestyle='--', color='b')
plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.6, 0.87, '95% cut-off threshold', color = 'red', fontsize=8)
plt.grid(axis='x')
plt.title('PCA without outliers', fontsize=14)
plt.xlabel('number of components', fontsize=8)
plt.ylabel('cumulative explained variance', fontsize=8)
plt.show()

In [ ]:
pca_1 = PCA(n_components=10).fit(X_scaled_no_outliers)
components = pca_1.fit_transform(X_scaled_no_outliers)
total_var = pca_1.explained_variance_ratio_.sum() * 100
n_components = len(pca_1.explained_variance_ratio_)

labels = {
    str(i): f"PC {i+1} ({var:.1f}%)"
    for i, var in enumerate(pca_1.explained_variance_ratio_ * 100)
}

fig = px.scatter_matrix(
    components,
    dimensions=range(n_components),
    labels=labels,
    title=f'Total Explained Variance: {total_var:.2f}%',
)

fig.update_traces(marker_size=1, diagonal_visible=False)

fig.update_layout(
    font=dict(size=8, color='black'),
    autosize=False,
    width=1100,
    height=1100,
)

fig.show()

In [ ]:
pca_10_outliers = PCA(n_components=10, random_state=RAND)
X_embedding_pca_10_outliers = pca_10_outliers.fit_transform(X_scaled_no_outliers)

fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'}
)
fig.update_traces(marker_size=2)
fig.show();

On the first graph (with outliers) they can be seen as isolated points far from the dense areas, and they could affect the calculation of PCA components by pulling them in the direction of the outliers. On the second graph we also observe outliers, but in less quantity, and the data seems more compact and dense. This cleaner dataset can improve the performance of clustering algorithms by focusing on the more consistent, central patterns within the data. But the difference is not so big, so we check clustering algorythms on the both datasets to compare results. 

#### K-Means Clustering Algorythm

##### Determining the Optimal Number of Clusters

In [ ]:
# We have dataset with 63000 data points, let's start from 200 clusters, we'll try to make as less as possible
k_max = 200

In [ ]:
# Define a dictionary to store the clustering parameters for different algorithms.
# For 'KMeans', we specify the 'random_state' parameter to ensure reproducibility of the results.
params_cluster = {
    'KMeans': {
        'random_state': RAND,
    },
    'AgglomerativeClustering': {
        
    }
}

Let's examine the data without outliers, because we are going to proceed with it. But we will also check this number of clusters on the data with outliers and compare quality of clustering.

In [ ]:
# Data without outliers
# counting inertia
clusters_pca_1 = []
losses_pca_1 = []
  
for k in range(k_max):
    model = KMeans(n_clusters=k+1, n_init='auto')
    model.fit(X_embedding_pca_10_outliers)
    clusters_pca_1.append(k+1)
    losses_pca_1.append(model.inertia_)   

In [ ]:
# Identifying the 'elbow point' in the inertia plot
plt.subplots(figsize=(6, 3))
plt.plot(clusters_pca_1, losses_pca_1)
plt.ylabel("Loss", fontsize=8)
plt.xlabel("Number of clusters", fontsize=8)
plt.title('Elbow method for the data without outliers', fontsize=10)
plt.show()

In [ ]:
# zooming the graph
plt.subplots(figsize=(6, 3))
plt.plot(clusters_pca_1, losses_pca_1)
plt.ylabel("Loss", fontsize=8)
plt.xlabel("Number of clusters", fontsize=8)
plt.xlim([0,70]);

Based on this elbow plot, we could choose 5-10 clusters. But let's see also the silhoette score for 2-11 clusters for the final decision.

In [ ]:
def cluster_silhouette(df, a, b):
    """
    Calculate and plot silhouette scores for different numbers of clusters to evaluate clustering performance.
    
    Parameters:
    - df: dataset to be clustered.
    - a: starting number of clusters.
    - b: ending number of clusters.
    """

    # Define the number of rows and columns for the subplot grid 
    n_clusters_range = b - a
    n_cols = 2  
    # Calculate the number of rows needed
    n_rows = (n_clusters_range + n_cols - 1) // n_cols  
    # Initialize the subplot grid
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 7 * n_rows))  

    # Ensure axes is always a 2D array to make a graph
    axes = np.array(axes).reshape(n_rows, n_cols)

    # Perform clustering and calculate silhouette scores
    for idx, n_clusters in enumerate(range(a, b)):
        ax = axes[idx // n_cols, idx % n_cols]  
        kmeans = KMeans(n_clusters=n_clusters, random_state=RAND, n_init=10)  
        cluster_labels = kmeans.fit_predict(df)
        silhouette_avg = silhouette_score(df, cluster_labels)   
        # Calculate the silhouette score for each sample
        sample_silhouette_values = silhouette_samples(df, cluster_labels)  

        y_lower = 10  
        # Plot the silhouette scores of each sample
        for i in range(n_clusters):
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]  
            ith_cluster_silhouette_values.sort()  # Sort the scores for better visualization

            size_cluster_i = ith_cluster_silhouette_values.shape[0]  # Get the number of samples in cluster i
            y_upper = y_lower + size_cluster_i  # Calculate the upper bound for the y-axis
            color = plt.cm.nipy_spectral(float(i) / n_clusters)  # Assign a color to the cluster

            # Fill the area under the silhouette curve
            ax.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, 
                             edgecolor=color, alpha=0.7)

            # Label the silhouette plots 
            ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontsize=12)

            y_lower = y_upper + 10  

        # Set the title, xlabel, and ylabel for the subplot
        ax.set_title(f"Silhouette Plot for n_clusters = {n_clusters}", fontsize=16)
        ax.set_xlabel("Silhouette Coefficient Value", fontsize=14)
        ax.set_ylabel("Cluster Number", fontsize=14)

        # Draw a vertical line to show the average silhouette score across all samples
        ax.axvline(x=silhouette_avg, color="red", linestyle="--")

        # Annotate the average silhouette score
        ax.text(silhouette_avg + 0.02, y_lower + 5, f"Silhouette = {silhouette_avg:.2f}", color="red", fontsize=12)

    # Remove any empty subplots 
    for i in range(idx + 1, len(axes.flat)):
        fig.delaxes(axes.flat[i])

    # Adjust the layout
    plt.tight_layout()
    plt.show()

In [ ]:
%%time
cluster_silhouette(X_embedding_pca_10_outliers, 2, 11)

With 2-6 clusters we observe the score 0.15, after it increse to 0.16, but it will be too many clusters that isn't matched with the idea of explainability, so let's think about dividing the data to 2-6 clusters. 

##### Clustering

Firstly, lets's see at the cluster's sizes. 

In [ ]:
def plot_size(data, labels, ax):
    """
    Plot the size of each cluster

    Parameters:
    data: the original dataset
    labels: cluster labels for each data point
    ax : matplotlib axes object to draw the plot on
    """
    # Adding cluster labels to the data
    cluster_data = data.assign(cluster=labels)

    # Counting the number of data points in each cluster
    cluster_counts = cluster_data['cluster'].value_counts().sort_index()

    # Creating a bar plot
    sns.barplot(x=cluster_counts.index, y=cluster_counts.values, ax=ax)
    ax.set_xlabel('Cluster Label')
    ax.set_ylabel('Number of Data Points')

In [ ]:
def plot_clustering_pca(data, 
                        data_scale, 
                        embedding, 
                        model: sklearn.base.ClusterMixin,
                        kwargs, 
                        min_size: int = 2, 
                        max_size: int = 6, 
                        type_train: str = 'embedding'):
    """
    Visualize cluster sizes using PCA-reduced data 
    
    Parameters:
    - data: the original dataframe for reference in plotting
    - data_scale: scaled data
    - embedding: PCA-reduced data 
    - model: clustering model from scikit-learn that implements ClusterMixin
    - kwargs: additional keyword arguments for the clustering model.
    - min_size: minimum number of clusters 
    - max_size: maximum number of clusters 
    - type_train: indicates whether to fit the model on 'data_scale' or 'embedding'.
    
    Returns:
    - dict_clusters: a dictionary with the number of clusters as keys and cluster labels as values.
    """
    
    dict_clusters = {}
    fig, axes = plt.subplots(1, max_size - min_size + 1, figsize=(16, 4))

    for i, clust in enumerate(range(min_size, max_size + 1)):
        # Ensure 'n_init' is explicitly set for KMeans models
        if 'KMeans' in str(model):
            n_init = kwargs.pop('n_init', 'auto')  
            clf = model(n_clusters=clust, n_init=n_init, **kwargs)
        else:
            clf = model(n_clusters=clust, **kwargs)

        # Fitting the model on the chosen data type
        if type_train == 'embedding':
            clf.fit(embedding)
        else:
            clf.fit(data_scale)

        # Saving the cluster labels in the dictionary
        dict_clusters[clust] = clf.labels_

        # Plotting the size of each cluster using the plot_size function written before
        ax = axes[i] if max_size - min_size > 0 else axes
        plot_size(data, dict_clusters[clust], ax=ax)

    # Setting the main title and adjust layout to prevent overlap of subplots
    plt.suptitle('Cluster Sizes for Different Number of Clusters', fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()

    return dict_clusters

Firsty we will have a look at the data with outliers.

In [ ]:
# Data with outliers
dict_clusters_pca = plot_clustering_pca(
    data=data_new,
    data_scale=X_scaled,
    embedding=X_embedding_pca_10,
    model=KMeans,
    kwargs=params_cluster['KMeans'],
    type_train='embedding'
)

In [ ]:
%%time
data_done_1 = data_new.copy()

for n_clusters in range(2, 7):  
    data_done_1 = data_done_1.assign(cluster_km=dict_clusters_pca[n_clusters])
    
    print(f'silhouette K-Means for {n_clusters} clusters: {round(silhouette_score(data_done_1, data_done_1.cluster_km), 2)}')

Now we will do the same for the data without outliers.

In [ ]:
# Data without outliers
dict_clusters_pca_outliers = plot_clustering_pca(
    data=data_no_outliers,
    data_scale=X_scaled_no_outliers,
    embedding=X_embedding_pca_10_outliers,
    model=KMeans,
    kwargs=params_cluster['KMeans'],
    type_train='embedding'
)

In [ ]:
%%time
data_done = data_no_outliers.copy()

for n_clusters in range(2, 7):  
    data_done = data_done.assign(cluster_km=dict_clusters_pca_outliers[n_clusters])
    
    print(f'silhouette K-Means for {n_clusters} clusters: {round(silhouette_score(data_done, data_done.cluster_km), 2)}')

As we can see, the data with outliers and without them is divided a bit differently, we could observe different cluster sizes and maybe more clear patterns. The silhoette score for the data without outliers is much higher, so we consider to observe clustering on the dataset without outliers. According to the highest silhouette score let's see divide to 2-4 clusters. 

In [ ]:
# Assigning columns to describe in analysis
object_cols = [
    'weather'
    
]

num_cols = [
    'siteID',
    'kWhDelivered',
    'temperature',
    'pressure',
    'charging_duration_disconnect',
    'charging_duration_done',
    'useless_parking',
    'connection_year', 
    'connection_month', 
    'connection_hour', 
    'disconnect_time', 
]

We write 3 functions which help us to analyse the clustering results. 

In [ ]:
def plotting_object(data, labels, object_cols):
    """
    Plot categorical features for different clusters, normalized by the size of each cluster.
    
    Parameters:
    - data: DataFrame containing the data.
    - labels: Series or array-like containing cluster labels for each row in `data`.
    - object_cols: List of column names in `data` that are categorical and should be plotted.
    """
    
    # Ensure labels are of string type to prevent issues with seaborn plotting
    labels = labels.astype(str)

    # Combine the dataset with cluster labels
    data_label = data.assign(cluster=labels)

    # Count the number of entries in each cluster
    size_cluster = data_label.groupby("cluster").size()

    # Determine the number of rows needed for subplots based on the number of categorical columns
    rows = len(object_cols)
    n_cols = 1  # Fixed number of columns for subplot grid

    # Create a subplot for each categorical column
    fig, axes = plt.subplots(nrows=rows, ncols=n_cols, figsize=(10, 6 * rows))

    # Ensure `axes` is an array even when there's only one subplot to maintain consistency
    if rows == 1:
        axes = np.array([axes])

    for num, col in enumerate(object_cols):
        # Calculate the normalized count for each category within each cluster
        data_norm = (
            data_label.groupby(["cluster", col]).size()
            / size_cluster
        ).reset_index(name='norm')
        
        # Sort categories by their overall frequency for better visualization
        order = data_norm.groupby(col)['norm'].sum().sort_values(ascending=False).index

        # Plot the normalized counts as bar plots
        sns.barplot(
            data=data_norm,
            y=col,  
            x='norm',  
            hue="cluster",  
            ax=axes[num],  
            order=order 
        )

        axes[num].set_xlabel('Normalized Count')
        axes[num].set_ylabel(col)
        axes[num].tick_params(axis='y', labelsize=10)
        axes[num].legend(title='Cluster', bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[num].set_title(f'Distribution of {col} by Cluster')

    # Adjust subplot layout to prevent overlapping and display the plot
    plt.tight_layout()
    plt.show()

In [ ]:
def plotting_kde_num(data, labels, num_cols):
    """
    Make Kernel Density Estimate plot for comparing numerical features 
    data: dataset
    labels: cluster labels
    num_cols: list of columns wuth numerical features
    """
    data_label = data.assign(cluster=labels)

    # Set the number of columns and rows for subplots
    n_cols = 3  
    n_rows = (len(num_cols) + n_cols - 1) // n_cols  # Calculate the necessary number of rows

    # Create subplots
    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(10, 4 * n_rows))
    axes = axes.flatten()  # Flatten to a one-dimensional array for convenience

    # Draw the plots
    for idx, col in enumerate(num_cols):
        if data_label[col].var() == 0:  # Check for zero variance
            axes[idx].text(0.5, 0.5, 'No variance in\n' + col, 
                           horizontalalignment='center', 
                           verticalalignment='center', 
                           transform=axes[idx].transAxes)
            axes[idx].set_title(col)
            axes[idx].set(xlabel='', ylabel='', xticklabels=[], yticklabels=[])
            continue  # Skip the rest of the loop and proceed with the next iteration

        sns.kdeplot(
            data=data_label,
            x=col,
            hue="cluster",
            fill=True,
            common_norm=False,
            palette="crest",
            alpha=0.4,
            linewidth=2,
            ax=axes[idx],
        )

    # Remove any empty subplots
    for i in range(len(num_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

In [ ]:
def plotting_num(data, labels, num_cols):
    """
    Make boxplot for comparing numerical features 
    data: dataset
    labels: cluster labels
    num_cols: list of columns wuth numerical features
    """
    data_label = data.assign(cluster=labels)

    n_cols = 3  
    n_rows = (len(num_cols) + n_cols - 1) // n_cols  

    fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(10, 5 * n_rows))
    axes = axes.flatten()  
   
    for idx, col in enumerate(num_cols):
        if data_label[col].var() == 0: 
            axes[idx].text(0.5, 0.5, 'No variance in\n' + col, 
                           horizontalalignment='center', 
                           verticalalignment='center', 
                           transform=axes[idx].transAxes)
            axes[idx].set_title(col)
            axes[idx].set(xlabel='', ylabel='', xticklabels=[], yticklabels=[])
            continue  
        
        sns.boxplot(
            data=data_label,
            y=col,
            x="cluster",
            palette="crest",
            ax=axes[idx],
        )

    for i in range(len(num_cols), len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.show()

Now we are ready to observe the clusters content.

##### 2 clusters 

In [ ]:
plotting_object(data_clusters_outliers, dict_clusters_pca_outliers[2], object_cols)
plotting_kde_num(data_clusters_outliers, dict_clusters_pca_outliers[2], num_cols)
plotting_num(data_clusters_outliers, dict_clusters_pca_outliers[2], num_cols)

In [ ]:
fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'},
    color=dict_clusters_pca_outliers[2]
)
fig.update_traces(marker_size=2)
fig.show()

**Cluster 0: Short Public Sessions**

Mostly site 2 (public), with medium or short kWh delivered (0-15). Parking without charging tends to be short, and the connection time is quite stable during the day (from 8 am to 3 pm), with disconnections mostly occurring around 6-7 pm. **People who use stations in public places occasionally for short top-ups.**

**Cluster 1: Long Private Sessions**

Primarily private sites (site 1), with medium to high energy delivered (20-25 kWh). Charging sessions and parking without charging are long (more than 4 hours). Connections occur very early (from 7 am to 10 am); after 10 am, new connections almost don't happen. Disconnections occur between 5 pm and 7 pm. **People who use charging stations close to work and charge from early morning throughout the working day.**

Value for business: helps in planning and optimizing charging infrastructure by ensuring that public stations are equipped for quick charges and are strategically located and private ones support longer charging durations. 

##### 3 clusters 

In [ ]:
plotting_object(data_clusters_outliers, dict_clusters_pca_outliers[3], object_cols)
plotting_kde_num(data_clusters_outliers, dict_clusters_pca_outliers[3], num_cols)
plotting_num(data_clusters_outliers, dict_clusters_pca_outliers[3], num_cols)

In [ ]:
fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'},
    color=dict_clusters_pca_outliers[3]
)
fig.update_traces(marker_size=2)
fig.show()

**Cluster 0: Flexible Working Chargers**

Site 1 (private), with a medium to large amount of energy delivered (10-20 kWh), and a wide range of session durations. There is a significant demand for connections at 8 am, but demand is lower throughout the day, with disconnections at 6 pm. **Connections are made close to the office during work hours to meet various energy demands.**

Value for business: usefull to plan tailored services such as dynamic pricing, reservation systems, or loyalty programs which could attract this user base. Also could plan some marketing targeted services to optimize utilization and increase revenue from these users.

**Cluster 1: First Working Users**

Site 1 (private), with a medium to large amount of energy delivered, featuring the longest session durations and parking times. More active in the early years of observation (2018-19). High demand in the morning (7-9 am), with disconnections between 4-6 pm. **First clients of the servise. Connections are made close to the office for the entire day, without time limitations.**

Value for business: efficient energy management and the opportunity to offer tailored energy packages or subscription models. Opportunities for off-peak energy management strategies to optimize grid load and reduce costs.

**Cluster 2: Public Chargers**

Mostly site 2 (public), with short to medium sessions and a medium amount of energy delivered. Parking duration is also short. Demand from 9 am to 4 pm, with disconnections mostly around 6-7 pm. **Connections in public places are for short durations for a quick energy boost**

Value for business: strategic placement of fast-charging stations in high-traffic public areas, mobile app integration for real-time charger availability and easy payment options. **

##### 4 clusters 

In [ ]:
plotting_object(data_clusters_outliers, dict_clusters_pca_outliers[4], object_cols)
plotting_kde_num(data_clusters_outliers, dict_clusters_pca_outliers[4], num_cols)
plotting_num(data_clusters_outliers, dict_clusters_pca_outliers[4], num_cols)

In [ ]:
fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'},
    color=dict_clusters_pca_outliers[4]
)
fig.update_traces(marker_size=2)
fig.show()

**Cluster 0: New Regular Consumers**

Exhibits three peaks for kWh delivered, around 5, 10, and 20 kWh. Charging duration is among the lowest, with mostly short parking without charging, but some long sessions are also present. Active from 2019-2021. Connections typically occur around 8 am, with disconnections at either 11 am or 7 pm. Charging occurs at both sites, but predominantly at Site 1 (private).  **Users charge close to work for long sessions, possibly to fully charge their batteries before going home.**

Value for business: time-based incentives and flexible pricing models, rewards for frequent use could encourage off-peak charging, helping to balance the grid's demand and increase station utilization during lower-demand periods. 

**Cluster 1: First and Flexible**

Characterized by low kWh delivery, mostly under 10 kWh, with occasional peaks around 20 kWh. Charging durations are among the longest, with the longest parking durations without charging (around 4 hours). Most active in the initial years, 2018-2019. Connections vary from 6 to 10 am, predominantly at 7 am, with disconnections between 5-6 pm, across both private and public sites. **Early adopters of the service, they prefer to fully charge, possibly at stations close to their work, homes, or places where they stay for extended periods**

Value for business: targeted promotions, such as loyalty rewards or discounts for consistent usage can enhance usage frequency and solidify their loyalty. Personalized communication highlighting new features or stations could further encourage engagement.

**Cluster 2: Charging Professionals**

Prefers charging at Site 1 (private), with high kWh deliveries (over 20), indicating either larger battery capacities or less frequent charging needs. Features the longest session durations, but short parking without charging times. Initially most active in 2018-2019, but also present in 2020-2021. Connections typically occur between 8-9 am, with disconnections from 5-6 pm.  **Professionals who rely on charging stations near their workplaces.** 

Value for business: tailored B2B solutions and partnerships, providing dedicated charging infrastructure or reserved charging slots for employees. Off-peak pricing models for long-term parking.

**Cluster 3: First Fast Public Users**

Primarily uses Site 2 (public), active in 2018-2019, with the shortest kWh deliveries and charging sessions, and very short parking times without charging. Connections occur at various times, notably at 9 am and 2-3 pm, with disconnections broadly spread but mostly around 6 pm. **Occasional daily users with diverse charging needs and routines.**

Value for business: targeted marketing with commercial fleets that could benefit from fast, public charging. Priority services, reserved spots during peak hours, or discounted rates for frequent users.

### AgglomerativeClustering

To determined the number of clusters let's have a look at the silhouette score.

In [ ]:
# Adapt the function which we used to plot the score for Agglomerative clustering
def cluster_silhouette_agglo(df, a, b):
    # Define the number of rows and columns for subplots
    n_clusters_range = b - a
    n_cols = 2
    n_rows = (n_clusters_range + n_cols - 1) // n_cols  # Calculate the number of rows
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 7 * n_rows))

    # Ensure axes is always a 2D array
    if n_rows == 1:
        axes = np.array(axes).reshape(-1, n_cols)
    elif n_cols == 1:
        axes = np.array(axes).reshape(n_rows, -1)

    for idx, n_clusters in enumerate(range(a, b)):
        ax = axes[idx // n_cols, idx % n_cols]
        model = AgglomerativeClustering(n_clusters=n_clusters)
        cluster_labels = model.fit_predict(df)

        silhouette_avg = silhouette_score(df, cluster_labels)
        sample_silhouette_values = silhouette_samples(df, cluster_labels)

        y_lower = 10
        for i in range(n_clusters):
            ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
            ith_cluster_silhouette_values.sort()

            size_cluster_i = ith_cluster_silhouette_values.shape[0]
            y_upper = y_lower + size_cluster_i
            color = plt.cm.nipy_spectral(float(i) / n_clusters)
            ax.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

            ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i), fontsize=12)

            y_lower = y_upper + 10

        ax.set_title(f"Silhouette Plot for n_clusters = {n_clusters}", fontsize=16)
        ax.set_xlabel("Silhouette Coefficient Value", fontsize=14)
        ax.set_ylabel("Cluster Number", fontsize=14)
        ax.axvline(x=silhouette_avg, color="red", linestyle="--")
        ax.text(silhouette_avg + 0.02, y_lower + 5, f"Silhouette = {silhouette_avg:.2f}", color="red", fontsize=12)

    # Remove any empty subplots
    for i in range(idx + 1, len(axes.flat)):
        fig.delaxes(axes.flat[i])

    plt.tight_layout()
    plt.show()

For explainability purposes, we prefer to limit the number of clusters to six. The final cluster count will be determined based on silhouette scores. Additionally, due to the higher computational demands of the agglomerative method, we aim to conserve resources by avoiding the creation of an excessive number of clusters.

In [ ]:
%%time
cluster_silhouette_agglo(X_embedding_pca_10_outliers, 2, 7)

The highest score we could reach with 5-6 clusters (0.12), so let's divide like this. 

In [ ]:
# Ajusted function fo agglomarative clustering
def plot_clustering_agglo(data, 
                          data_scale, 
                          embedding, 
                          kwargs,
                          min_size=5, 
                          max_size=6, 
                          type_train='embedding'):
    """
    Visualize cluster sizes using PCA-reduced data with Agglomerative Clustering.
    
    Parameters:
    - data: the original dataframe for reference in plotting.
    - data_scale: scaled data.
    - embedding: PCA-reduced data.
    - kwargs: additional keyword arguments for the clustering model.
    - min_size: minimum number of clusters.
    - max_size: maximum number of clusters.
    - type_train: indicates whether to fit the model on 'data_scale' or 'embedding'.
    
    Returns:
    - dict_clusters: a dictionary with the number of clusters as keys and cluster labels as values.
    """
    
    dict_clusters = {}
    fig, axes = plt.subplots(1, max_size - min_size + 1, figsize=(16, 4))
    
    for i, clust in enumerate(range(min_size, max_size + 1)):
        # Initialize the AgglomerativeClustering model
        clf = AgglomerativeClustering(n_clusters=clust, **kwargs)
        
        # Fit the model on the chosen data type
        labels = clf.fit_predict(embedding if type_train == 'embedding' else data_scale)
        
        # Save the cluster labels in the dictionary
        dict_clusters[clust] = labels

        # Plotting the size of each cluster
        ax = axes[i] if max_size - min_size > 0 else axes
        plot_size(data, dict_clusters[clust], ax=ax)
    
    # Setting the main title and adjust layout to prevent overlap of subplots
    plt.suptitle('Cluster Sizes for Different Number of Clusters', fontsize=14)
    plt.tight_layout()
    plt.subplots_adjust(top=0.85)
    plt.show()
    
    return dict_clusters

Let's see the cluster sizes.

In [ ]:
%%time

dict_clusters_agglo = plot_clustering_agglo(
    data=data_no_outliers,
    data_scale=X_scaled_no_outliers,
    embedding=X_embedding_pca_10_outliers,
    kwargs=params_cluster['AgglomerativeClustering'],
    type_train='embedding'
)

In [ ]:
def plot_dendrogram(model, **kwargs):

    # Children of hierarchical clustering
    children = model.children_

    # Distances between each pair of children
    # Since we don't have this information, we can use a uniform one for plotting
    distance = np.arange(children.shape[0])

    # The number of observations contained in each cluster level
    no_of_observations = np.arange(2, children.shape[0]+2)

    # Create linkage matrix and then plot the dendrogram
    linkage_matrix = np.column_stack([children, distance, no_of_observations]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

##### 5 clusters 

In [ ]:
%%time
agglo_5 = AgglomerativeClustering(n_clusters=5) 
pred_agglo_5 = agglo_5.fit_predict(X_embedding_pca_10_outliers)

In [ ]:
%%time
# Dendrogram
plt.figure(figsize=(15,10))
plt.title('Hierarchical Clustering Dendrogram for 5 Clusters')
plot_dendrogram(agglo_5, labels=dict_clusters_agglo[5])
plt.ylabel("Distance")
plt.show()

The dendrogram seems quite complex. To see the connections between data points we will make another truncated dendrogram to focus on the most significant cluster mergers in hierarchical clustering.

In [ ]:
# Using the ward method which minimizes the variance of the clusters being merged
Z = linkage(X_embedding_pca_10_outliers, 'ward')

plt.figure(figsize=(8, 6))
plt.title('Hierarchical Clustering Dendrogram for 5 Clusters')

# Determine the color threshold to differentiate clusters
color_threshold = Z[-5, 2] - 0.01  # Subtracting a small value to ensure the threshold is just below the merge point.

# Generate the dendrogram plot.
dendrogram(
    Z,
    truncate_mode='lastp',    # 'lastp' truncation mode shows only the last 'p' merged clusters.
    p=5,                     
    leaf_rotation=90.,        # Rotate the leaf labels (x-axis labels) to avoid overlap and improve readability.
    leaf_font_size=8.,        
    show_contracted=True,     # Enable a condensed view to provide an overview of the larger clusters' structure.
    color_threshold=color_threshold  # Apply the calculated color threshold to distinguish clusters by color.
)

plt.ylabel("Distance")
plt.show()

Now we wil see the content of the clusters.

In [ ]:
plotting_object(data_clusters_outliers, dict_clusters_agglo[5], object_cols)
plotting_kde_num(data_clusters_outliers, dict_clusters_agglo[5], num_cols)
plotting_num(data_clusters_outliers, dict_clusters_agglo[5], num_cols)

In [ ]:
fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'},
    color=dict_clusters_agglo[5]
)
fig.update_traces(marker_size=2)
fig.show()

**Cluster 0: First Working Clients** 

Mostly fair weather, private and public places in 2018-19. Highest kWh delivered (25 and 40), longest charging duration, but using station without charging is short. Connection through all day but mostly at 8 am, disconnection at 6 pm, but could be also at night. 

Value for business: subscription-based plans for guaranteed access to charging spots, especially during peak hours. Providing workplace charging or partnering with employers could ensure loyalty and regular usage.

**Cluster 1: First Public Occasional Clients** 

Site 2 (public) in 2018-1019, lowest kWh delivered (up to 10), lowest session's duration and parking time, connection time spread throught all day with light peaks for 9 am, 3-4 pm, disconnection at 7 pm. 

Value for business: engagement methods like discounted rates during off-peak hours or loyalty points for each session.

**Cluster 2: Late Workers** 

Variance of cloudy, rain, smoke. Mostly site 1 (private) in 2018-19, medium kWh delivered, connection since 11-12 am till 5 pm, there are some cases with parking without charging.

Value for business: offerings like late charging discounts or reserved spots, increasing station utilization during off-peak hours.

**Cluster 3: Modern Working Clients** 

Highest occurence of the fair weather. Site 1 (private), medium kWh delivered, active mostly in 2021, charging fromm 8 am till 5-6pm. 

Value for business: services that align with a regular work schedule, such as reserved parking with charging during work hours. 

**Cluster 4: Full Chargers** 

Charging at both sites, medium and high energy delivered (10-20 kWh), longest parking without charging (4 hours), earliest sessions from 5 am till 9 am, disconnection around 5-6 pm. 

Value for business: priority services, such as dedicated charging areas and faster chargers, can ensure that these high-usage clients maintain a high level of satisfaction. Dynamic pricing structure that rewards higher consumption could also incentivize continued loyalty.

##### 6 clusters

In [ ]:
%%time
agglo_6 = AgglomerativeClustering(n_clusters=6) #The number of clusters to find
pred_agglo_6 = agglo_6.fit_predict(X_embedding_pca_10_outliers)

In [ ]:
%%time
# Dendrogram
plt.figure(figsize=(15,10))
plt.title('Hierarchical Clustering Dendrogram for 6 Clusters')
plot_dendrogram(agglo_6, labels=dict_clusters_agglo[5])
plt.ylabel("Distance")
plt.show()

In [ ]:
%%time
# Truncated dendrogram
Z = linkage(X_embedding_pca_10_outliers, 'ward')

plt.figure(figsize=(8, 6))
plt.title('Hierarchical Clustering Dendrogram for 6 Clusters')

color_threshold = Z[-5, 2] - 0.01  # Subtracting a small value to ensure the threshold is just below the merge point.

dendrogram(
    Z,
    truncate_mode='lastp',   
    p=6,                     
    leaf_rotation=90.,        
    leaf_font_size=8.,        
    show_contracted=True,      
    color_threshold=color_threshold  
)

plt.ylabel("Distance")
plt.show()

In [ ]:
plotting_object(data_clusters_outliers, dict_clusters_agglo[6], object_cols)
plotting_kde_num(data_clusters_outliers, dict_clusters_agglo[6], num_cols)
plotting_num(data_clusters_outliers, dict_clusters_agglo[6], num_cols)

In [ ]:
fig = px.scatter_3d(
    X_embedding_pca_10_outliers, x=0, y=1, z=2,
    labels={'color': 'species'},
    color=dict_clusters_agglo[6]
)
fig.update_traces(marker_size=2)
fig.show()

**Cluster 0: First Stable Clients** 

Mostly fair weather. Private and public places. High range of the delivered energy (0-20 kWh), but the shortest charging and parking duration. Active in 2018-19. Stable connection demand since 8 am till 5 pm, disconnection at 7 pm. **Mid-duration charge, behavior fits within a standard workday**

**Cluster 1: Long Day Parkers** 

Medium energy delivered, medium charging duration, but longest parking without charging (more than 4 hours). Connection 5-6 am, disconnection 4-8 pm. **They leave a car in the morning for all day not chacking the charging session, park for longer than it needed**

**Cluster 2: Fast Day Chargers** 

Most different weather (variance of "cloudy", smoke, storm). Medium energy delivered, shortest parking duration, connection around 11 am. **Prefer to make fast boost of energy in the morning**

**Cluster 3: Top-up Close To Office**

Highest occurence of the fair weather. Medium energy delivered (around 20 kWh), occurs mostly in 2021, private place (1), connection and disconnection around 11 am, possibly very short charging. **Charging close to job but not typically long**

**Cluster 4: Long Chargers** 

Highest energy delivered (around 40 kWh), longest charging sessions. **Those who prefer full charge or prepare for a long day driving**

**Cluster 5: Night Public Chargers** 

Mostly site 2 (public), medium energy consumption (25 kWh), stable long parking without charging (could be 6 hours). Connection hour around 9 pm, disconnection at 1 am or 7 am. **Those who leave a car for a several hours at night at the public place**

#### Conclusion of the Cluster Analysis

Based on the cluster descriptions and silhouette score, for K-means clustering with 4 clusters appears to be more explanatory. For Agglomerative clustering we prefered 5 clusters result. Let's compare the results of both algorythms. 

In [ ]:
# Adding labels of the best clustering results
data_fin = data_no_outliers.copy()
data_fin = data_fin.assign(cluster_km=dict_clusters_pca_outliers[4], 
                           cluster_ag=dict_clusters_agglo[5])

data_fin.tail(2)

In [ ]:
%%time
# Checking the silhouette score
print(
    f'silhouette K-Means: {round(silhouette_score(data_fin, data_fin.cluster_km), 2)}'
)
print(
    f'silhouette Agl: {round(silhouette_score(data_fin, data_fin.cluster_ag), 2)}'
)

A silhouette score of 0.21 for the K-Means clustering algorithm suggests that on average, objects are reasonably well clustered. A silhouette score of 0.06 for the Agglomerative Clustering algorithm suggests that the clusters are quite weak and there is considerable overlap between them. K-Means algorithm performs better.

In [ ]:
# Calculating the Davies-Bouldin Index
db_kmeans = davies_bouldin_score(data_fin, data_fin.cluster_km)
print(f'Davies-Bouldin K-Means: {round(db_kmeans, 2)}')

db_agl = davies_bouldin_score(data_fin, data_fin.cluster_ag)
print(f'Davies-Bouldin Agglomerative: {round(db_agl, 2)}')

A Davies-Bouldin index of 1.61 for the K-Means clustering suggests that the clusters are moderately well-separated and compact, but there's room for improvement. A Davies-Bouldin index for Agglomerative Clustering suggests that the clusters created by this method are either very close to each other or very dispersed. 

In [ ]:
# Calculating the Calinski-Harabasz Index
ch_kmeans = calinski_harabasz_score(data_fin, data_fin.cluster_km)
print(f'Calinski-Harabasz K-Means: {round(ch_kmeans, 2)}')

ch_agl = calinski_harabasz_score(data_fin, data_fin.cluster_ag)
print(f'Calinski-Harabasz Agglomerative: {round(ch_agl, 2)}')

A Calinski-Harabasz index of 27,743.29 for K-Means is quite high, suggesting that the clusters are distinct and well-separated from each other, with tight clustering within each cluster. For Agglomerative Clustering the score is lower compared to K-Means, which indicates that the clusters it formed are less dense and less well-separated. 

Overall, K-Means clustering algorithm outperforms Agglomerative Clustering for this dataset across all three metrics.

#### Final Conclusion

Using K-Means algorythm we found these typical patterns in user behaviour. Also we provide some business recomendations for every target group.

**Cluster 0: New Regular Consumers**

Exhibits three peaks for kWh delivered, around 5, 10, and 20 kWh. Charging duration is among the lowest, with mostly short parking without charging, but some long sessions are also present. Active from 2019-2021. Connections typically occur around 8 am, with disconnections at either 11 am or 7 pm. Charging occurs at both sites, but predominantly at Site 1 (private). **Users charge close to work for long sessions, possibly to fully charge their batteries before going home.**

Value for business: time-based incentives and flexible pricing models, rewards for frequent use could encourage off-peak charging, helping to balance the grid's demand and increase station utilization during lower-demand periods.

**Cluster 1: First and Flexible**

Characterized by low kWh delivery, mostly under 10 kWh, with occasional peaks around 20 kWh. Charging durations are among the longest, with the longest parking durations without charging (around 4 hours). Most active in the initial years, 2018-2019. Connections vary from 6 to 10 am, predominantly at 7 am, with disconnections between 5-6 pm, across both private and public sites. **Early adopters of the service, they prefer to fully charge, possibly at stations close to their work, homes, or places where they stay for extended periods**

Value for business: targeted promotions, such as loyalty rewards or discounts for consistent usage can enhance usage frequency and solidify their loyalty. Personalized communication highlighting new features or stations could further encourage engagement.

**Cluster 2: Charging Professionals**

Prefers charging at Site 1 (private), with high kWh deliveries (over 20), indicating either larger battery capacities or less frequent charging needs. Features the longest session durations, but short parking without charging times. Initially most active in 2018-2019, but also present in 2020-2021. **Connections typically occur between 8-9 am, with disconnections from 5-6 pm. Professionals who rely on charging stations near their workplaces.**

Value for business: tailored B2B solutions and partnerships, providing dedicated charging infrastructure or reserved charging slots for employees. Off-peak pricing models for long-term parking.

**Cluster 3: First Fast Public Users**

Primarily uses Site 2 (public), active in 2018-2019, with the shortest kWh deliveries and charging sessions, and very short parking times without charging. Connections occur at various times, notably at 9 am and 2-3 pm, with disconnections broadly spread but mostly around 6 pm. **Occasional daily users with diverse charging needs and routines.**

Value for business: targeted marketing with commercial fleets that could benefit from fast, public charging. Priority services, reserved spots during peak hours, or discounted rates for frequent users.